In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Imputer, LabelBinarizer, Normalizer, OneHotEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression
import xgboost
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, RandomForestRegressor, AdaBoostRegressor
from sklearn.model_selection import GridSearchCV, ShuffleSplit
pd.options.display.max_columns = None
import warnings
warnings.filterwarnings("ignore")
import lightgbm as lgb

In [2]:
# 检查列缺失值，返回缺失值百分比的字典
def check_nan(df_var):
    print(df_var.shape)
    nan_result = df_var.isnull().sum(axis=0)
    col_name_list = df_var.columns.values
    result_dict = {k: v for k, v in zip(col_name_list, list(nan_result))}

    total = df_var.shape[0]
    
    nan_dict = dict()
    for rd in result_dict.items():
        if rd[1] != 0:
            print("{}: {}%".format(rd[0], round((rd[1]/total)*100, 2)))
            nan_dict[rd[0]] = round((rd[1]/total)*100, 2)

    return nan_dict

def check_row_nan_count(df_var):
    row_null = df_var.isnull().sum(axis=1)
    row_null_df = pd.DataFrame(df_var["ID"])
    row_null_df["count"] = row_null
    
    total_columns = df_var.shape[1]
    row_null_df["rate"] = 0
    rate_list = []
    for index, rn in enumerate(row_null_df["count"]):
        rate_list.append(round(rn/total_columns, 2))
    row_null_df["rate"] = rate_list
    
    print(row_null_df.groupby("rate")["count"].value_counts())
    return row_null_df

# 箱线图 连续类型特征（多选），枚举类型特征（单选）
def draw_box(df, value_fields=None, y_col=None, have_y=False):

    for value_field in value_fields:
        # data = pd.concat([df[value_field], df[y_col]], axis=1)
        plt.subplots(figsize=(15, 12))
        if have_y:
            fig = sns.boxplot(x=df[y_col], y=value_field, data=df)
        else:
            fig = sns.boxplot(x=None, y=value_field, data=df)
        plt.xlabel(y_col, fontdict={'weight': 'normal', 'size': 24})
        plt.ylabel(value_field, fontdict={'weight': 'normal', 'size': 24})
        plt.yticks(size=18)
        plt.xticks(size=18)
        fig.axis()
        plt.show()

# 填补缺失值 mean median most_frequent constant
def imputer_nan(df, axis=None, cols=None, missing_values="NaN", strategy='mean', fill_value=0):
    if strategy == "constant":
        if cols:
            part_a = df.drop(cols, axis=axis)
            part_b = df[cols]
            try:
                fill_value = float(fill_value)
            except TypeError:
                pass

            part_b = part_b.fillna(fill_value)
            # 合并
            df = pd.concat([part_a, part_b], axis=axis)
        else:
            df = df.fillna(fill_value)
    else:
        imp = Imputer(missing_values=missing_values, strategy=strategy)
        try:
            if cols:
                part_a = df.drop(cols, axis=axis)
                part_b = df[cols]
                if "object" in str(part_b.dtypes):
                    print("is object")
                part_b_name_list = part_b.columns.values
                part_b = imp.fit_transform(part_b)
                part_b = pd.DataFrame(part_b, columns=part_b_name_list)
                # 合并
                df = pd.concat([part_a, part_b], axis=axis)
            else:
                col_name_list = df.columns.values
                df = imp.fit_transform(df)
                df = pd.DataFrame(df, columns=col_name_list)
        except ValueError as e:
            print("部分列中存在多种类型的数据，请先转哑变量后再填充")
            raise e

    return df

# xgboost分类
def xgboost_classifier(x, y, max_depth=3, learning_rate=0.1, n_estimators=100, min_child_weight=1, gamma=0,
                                  subsample=1, colsample_bytree=1, scale_pos_weight=1, random_state=27, reg_alpha=0, reg_lambda=1):

    if max_depth:
        max_depth = int(max_depth)
        if max_depth < 0:
            max_depth = None
    else:
        max_depth = None

    learning_rate = float(learning_rate)
    n_estimators = int(n_estimators)
    min_child_weight = int(min_child_weight)
    gamma = float(gamma)
    subsample = float(subsample)
    colsample_bytree = float(colsample_bytree)
    scale_pos_weight = int(scale_pos_weight)

    random_state = int(random_state)

    # 拟合XGBoost模型
    model = xgboost.XGBClassifier(max_depth=max_depth, learning_rate=learning_rate, n_estimators=n_estimators,
                                  min_child_weight=min_child_weight, gamma=gamma,
                                  subsample=subsample, colsample_bytree=colsample_bytree,
                                  scale_pos_weight=scale_pos_weight, random_state=random_state, reg_alpha=reg_alpha, reg_lambda=reg_lambda)

    model.fit(x, y)
    return model

# 划分训练集和测试集
def split_train_test(df, random_state=33, test_size=0.25):
    random_state = int(random_state)
    test_size = float(test_size)
    train, test = train_test_split(df, random_state=random_state, test_size=test_size)
    return train, test

def split_column(df, y="y"):
    try:
        X = df.drop(y, axis=1)
    except KeyError:
        raise KeyError("请在拆分列的参数中选择数据中有的字段")
    y = pd.DataFrame(df[y], columns=[y])
    return X, y

# 类别分析(分析分类的占比的分布)
def check_classifier(df, col_name="Y"):
    result = df[col_name].value_counts()
    cols = result.index
    result_dict = {k: v for k, v in zip(cols, list(result))}
    print(result_dict)
    print("1:0 => {}:{}".format(result_dict[1], result_dict[0]))
    print("比例: {}".format(round(result_dict[0]/result_dict[1], 1)))
    return None

def logistic_regression(x, y, penalty="l2", class_weight=None, C=1):
    # if float(C) < 0 or float(C) > 1:
    #     C = 1
    try:
        if float(C) < 0 or float(C) > 1:
            C = 1
    except ValueError:
        C = 1

    if class_weight in ["", None]:
        class_weight = None
    else:
        class_weight = eval(class_weight)

    logic = LogisticRegression(penalty=penalty, class_weight=class_weight, C=C)
    logic.fit(x, y)

    return logic

# 归一化
def min_max_scaler(df, target=None, min_value=0, max_value=1):
    max_value = float(max_value)
    min_value = float(min_value)

    if not target:
        target = list(df.columns.values)
    maxmin = MinMaxScaler(feature_range=[min_value, max_value])
    for col in target:
        df[col] = maxmin.fit_transform(df[col].values.reshape(-1, 1))
    return df

# 标准化
def standard(df):
    col_name_list = df.columns.values
    std = StandardScaler()
    df = std.fit_transform(df)
    df = pd.DataFrame(df, columns=col_name_list)
    return df

# SVM分类
def svm_classifier(x, y, kernel='rbf', C=1, gamma=0.1, degree=3, class_weight=None, coef0=0):
    try:
        if float(C) < 0:
            C = 1
        else:
            C = float(C)
    except ValueError:
        C = 1

    try:
        if float(gamma) < 0:
            gamma = 0.1
        else:
            gamma = float(gamma)
    except ValueError:
        gamma = 0.1

    if gamma == "":
        gamma = 0.1

    try:
        if int(degree) < 0:
            degree = 3
    except ValueError:
        degree = 3
    degree = int(degree)
    try:
        coef0 = float(coef0)
    except ValueError:
        coef0 = 0

    if class_weight in ["", None]:
        class_weight = None
    else:
        class_weight = eval(class_weight)

    clf = svm.SVC(C=C, cache_size=200, class_weight=class_weight, coef0=coef0, decision_function_shape=None,
                  degree=degree,
                  gamma=gamma, kernel=kernel, max_iter=-1, probability=True, random_state=None, shrinking=True,
                  tol=0.001, verbose=False)
    clf.fit(x, y)

    return clf

# 主成分分析-降维
def pca(df, n_components="mle"):

    # col_name_list = df.columns.values

    try:
        n_components = int(n_components)
    except ValueError:
        pass

    pa = PCA(n_components=n_components)
    df = pa.fit_transform(df)

    # df = pd.DataFrame(df, columns=col_name_list)
    return df

# 随机森林分类
def random_forest_classifier(x, y, criterion="gini", n_estimators=10, max_depth=10, min_samples_split=2,
                             min_samples_leaf=1, min_weight_fraction_leaf=0., max_features=None, random_state=None,
                             max_leaf_nodes=None, min_impurity_decrease=0., min_impurity_split=None):
    if max_depth:
        max_depth = int(max_depth)
        if max_depth < 0:
            max_depth = None
    else:
        max_depth = None

    n_estimators = int(n_estimators)

    if "." in str(min_samples_split):
        min_samples_split = float(min_samples_split)
    else:
        min_samples_split = int(min_samples_split)

    if "." in str(min_samples_leaf):
        min_samples_leaf = float(min_samples_leaf)
    else:
        min_samples_leaf = int(min_samples_leaf)

    if max_features in ["auto", "sqrt", "log2", None]:
        pass
    else:
        if "." in str(max_features):
            max_features = float(max_features)
        else:
            max_features = int(max_features)

    random_state = int(random_state)

    if max_leaf_nodes:
        max_leaf_nodes = int(max_leaf_nodes)

    min_weight_fraction_leaf = float(min_weight_fraction_leaf)
    min_impurity_decrease = float(min_impurity_decrease)
    min_impurity_split = float(min_impurity_split)

    # estimator
    rf = RandomForestClassifier(criterion=criterion, n_estimators=n_estimators,
                                max_depth=max_depth, min_samples_split=min_samples_split,
                                min_samples_leaf=min_samples_leaf,
                                min_weight_fraction_leaf=min_weight_fraction_leaf, max_features=max_features,
                                random_state=random_state, max_leaf_nodes=max_leaf_nodes,
                                min_impurity_decrease=min_impurity_decrease, min_impurity_split=min_impurity_split)
    rf.fit(x, y)
    return rf

# 变量影响因子(仅仅针对决策树和随机森林)(特征重要性)
def get_importance(x, y, criterion="gini", splitter="best", max_depth=10, min_samples_split=2,
                     min_samples_leaf=1, min_weight_fraction_leaf=0., max_features=None, random_state=27,
                     max_leaf_nodes=None, min_impurity_decrease=0., min_impurity_split=None, presort=False,
                     node_id=None):

    if max_depth:
        max_depth = int(max_depth)
        if max_depth < 0:
            max_depth = None
    else:
        max_depth = None

    if "." in str(min_samples_split):
        min_samples_split = float(min_samples_split)
    else:
        min_samples_split = int(min_samples_split)

    if "." in str(min_samples_leaf):
        min_samples_leaf = float(min_samples_leaf)
    else:
        min_samples_leaf = int(min_samples_leaf)

    if max_features in ["auto", "sqrt", "log2", None]:
        pass
    else:
        if "." in str(max_features):
            max_features = float(max_features)
        else:
            max_features = int(max_features)

    random_state = int(random_state)

    if max_leaf_nodes:
        max_leaf_nodes = int(max_leaf_nodes)

    min_weight_fraction_leaf = float(min_weight_fraction_leaf)
    min_impurity_decrease = float(min_impurity_decrease)
    min_impurity_split = float(min_impurity_split)

    dec = DecisionTreeClassifier(criterion=criterion, splitter=splitter, max_depth=max_depth,
                                 min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf,
                                 min_weight_fraction_leaf=min_weight_fraction_leaf, max_features=max_features,
                                 random_state=random_state, max_leaf_nodes=max_leaf_nodes,
                                 min_impurity_decrease=min_impurity_decrease, min_impurity_split=min_impurity_split,
                                 presort=presort)

    try:
        dec.fit(x, y)
    except ValueError:
        raise ValueError("只有离散的标签才可以使用此算子")
    result_dict = {k: v for k, v in zip(x.columns.values, list(dec.feature_importances_))}
    dict_list = sorted(result_dict.items(), key=lambda x: x[1], reverse=True)

    items = []
    header = ["列名称", "影响因子"]
    items.append(header)
    for k, v in dict_list:
        item = collections.OrderedDict()
        item["列名称"] = k
        item["影响因子"] = v
        items.append(item)

    # 保存运算结果到数据库
    save_node_run_result(node_id, items)

    return x, y

# 删除列
def delete_column(df, target=None, inverse=False):
    # 默认值
    if target is None:
        target = []
    # 所有列
    col_name_list = list(df.columns.values)
    # 两种情况
    if inverse:
        for t in target:
            col_name_list.remove(t)
        df = df.drop(col_name_list, axis=1)
    else:
        df = df.drop(target, axis=1)
    return df

# lgbm分类
def lgbm_classifier(x, y, max_depth=3, learning_rate=0.1, n_estimators=100, min_child_weight=1, gamma=0,
                                  subsample=1, colsample_bytree=1, scale_pos_weight=1, random_state=27):

    if max_depth:
        max_depth = int(max_depth)
        if max_depth < 0:
            max_depth = None
    else:
        max_depth = None

    learning_rate = float(learning_rate)
    n_estimators = int(n_estimators)
    min_child_weight = int(min_child_weight)
    gamma = float(gamma)
    subsample = int(subsample)
    colsample_bytree = float(colsample_bytree)
    scale_pos_weight = int(scale_pos_weight)
    random_state = int(random_state)

    model = lgb.LGBMClassifier(max_depth=max_depth, learning_rate=learning_rate, n_estimators=n_estimators,
                                  min_child_weight=min_child_weight, gamma=gamma,
                                  subsample=subsample, colsample_bytree=colsample_bytree,
                                  scale_pos_weight=scale_pos_weight, random_state=random_state)
    model.fit(x, y)

    return model

# 处理时间格式数据
# 00:00 ==> 3800
# NaN   ==> 0
def tran_date(df_var, field_name):
    date_value = []
    for dlt in df_var[field_name]:
        if not isinstance(dlt, float):
            a, b = dlt.split(":")
            value = float(a) * 60 + float(b)
            date_value.append(3800 - value)
        else:
            date_value.append(0)
    df_var[field_name] = date_value
    return df_var

# 转换为one-hot编码
def OnehotEncoding(df, columns=None):
    if not columns:
        col_name_list = df.columns.values

        obj_list = []
        for index, d in enumerate(df.dtypes):
            if d == "object":
                obj_list.append(col_name_list[index])

        columns = obj_list

    data = df[columns]
    # 实例化OnehotEncoder
    enc = OneHotEncoder(categories="auto")
    # 生成目标特征列One_hot编码
    data_encoded = enc.fit_transform(data).toarray()
    # 生成新的列名
    new_columns = list(enc.get_feature_names())
    for i, column_name in enumerate(new_columns):
        df[column_name] = data_encoded[:, i]
    return df

# 自动分箱分析（卡方分箱，需要标签值，需要选定分成几个箱子）（展示分段）
def Chi_merge(df, x_col=None, y_col=None, k=6, rate=0.4):
    if not x_col:
        x_col = list(df.columns.values)

    pinf = float('inf')  # 正无穷大
    ninf = float('-inf')  # 负无穷大
    df_json = []
    
    bin_dict = dict()
    for col in x_col:
        print(col)
        col_result_dict = {}
        # 读取数据
        data = pd.concat([df[col], df[y_col]], axis=1)

        # 数据统计[feature1, class1, class2, class3]
        df_count = data.groupby(col)[y_col].value_counts().unstack().reset_index()
        df_count.fillna(0, inplace=True)

        # 去掉一些行，用于加快计算速度
        while df_count.shape[0] > 120:
            drop_row_index = []
            threshold = (data.shape[0] / df_count.shape[0]) * rate
            for i in range(df_count.shape[0] - 1):
                diff = 0
                for j in range(df_count.shape[1] - 1):
                    diff = abs(df_count.iloc[i, j + 1] - df_count.iloc[i + 1, j + 1]) + diff
                if diff < threshold:
                    drop_row_index.append(i + 1)
            df_count.drop(drop_row_index, inplace=True)
            len_df_count = df_count.shape[0]
            df_count.index = [i for i in range(len_df_count)]
            # print(len_df_count)

        df_count.index.name = 'index'
        n_class = df_count.shape[1] - 1
        num_interval = df_count.shape[0]
        print("num_interval: ", num_interval)
        max_col = pinf

        # 计算卡方值，合并删除，直到行数为k
        while (num_interval > k):
            chi_values = []
            drop_index = []
            for i in range(num_interval - 1):  # 制作表格，行列分别求和，用于计算卡方值
                data_chi = df_count.iloc[i:i + 2, 1:].copy().reset_index(drop=True)
                data_chi['sum'] = data_chi.apply(lambda x: x.sum(), axis=1)
                data_chi.loc[2] = data_chi.apply(lambda x: x.sum())
                for index in range(2):
                    for j in range(n_class):  # 计算卡方值
                        data_chi.iloc[index, j] = (data_chi.iloc[index, j] -
                                                   data_chi.iloc[index, -1] * data_chi.iloc[2, j] / data_chi.iloc[
                                                       2, -1]) ** 2 / \
                                                  (data_chi.iloc[index, -1] * data_chi.iloc[2, j] / data_chi.iloc[
                                                      2, -1])
                chi_value = sum(data_chi.iloc[0:-1, 0:-1].sum())
                chi_values.append(chi_value)
            min_chi = min(chi_values)  # 最小卡方值
            for i in range(num_interval - 2, -1, -1):  # 合并最小卡方值的行
                if chi_values[i] == min_chi:
                    df_count.iloc[i, 1:] = df_count.iloc[i, 1:] + df_count.iloc[i + 1, 1:]
                    drop_index.append(i + 1)
            df_count.drop(drop_index, inplace=True)  # 丢弃最小卡方值的行
            num_interval = df_count.shape[0]
            df_count.index = [i for i in range(num_interval)]
            df_count.iloc[0, 0] = ninf

        binning_list = list(df_count[col])
        binning_list.remove(binning_list[0])
        print(binning_list)
        
        bin_dict[col] = binning_list
    return bin_dict


def train_and_valid(train_x, train_y, valid_x, valid_y, n_estimators=88, max_depth=8, gamma=0, learning_rate=0.1, reg_alpha=0, reg_lambda=1, random_state=27):
    # xgboost分类
    xg_model = xgboost_classifier(train_x, train_y, max_depth=max_depth, learning_rate=learning_rate, n_estimators=n_estimators, min_child_weight=1, gamma=gamma, subsample=1, colsample_bytree=1, scale_pos_weight=1, random_state=random_state, reg_alpha=reg_alpha, reg_lambda=reg_lambda)
    # 逻辑回归
#     xg_model = logistic_regression(train_x, train_y, penalty="l2", class_weight=None, C=1)
    # 支持向量机
#     xg_model = svm_classifier(train_x, train_y, kernel='rbf', C=1, gamma=0.1, degree=3, class_weight=None, coef0=0)
    # 随机森林
#     xg_model = random_forest_classifier(train_x, train_y, criterion="gini", n_estimators=50, max_depth=7, min_samples_split=2, min_samples_leaf=2, min_weight_fraction_leaf=0., max_features="log2", random_state=33, max_leaf_nodes=None, min_impurity_decrease=0., min_impurity_split=1e-7)
    # lgbm分类
#     xg_model = lgbm_classifier(train_x, train_y, max_depth=8, learning_rate=0.1, n_estimators=100, min_child_weight=1, gamma=0, subsample=1, colsample_bytree=1, scale_pos_weight=1, random_state=27)
    
    y_pred = xg_model.predict(valid_x)

    accuracy_score_result = metrics.accuracy_score(valid_y, y_pred)
    precision_score_result = metrics.precision_score(valid_y, y_pred, average='macro')
    recall_score_result = metrics.recall_score(valid_y, y_pred, average='macro')
    f1_score_result = metrics.f1_score(valid_y, y_pred, average='macro')

#     print("accuracy_score_result: {}".format(accuracy_score_result))
#     print("precision_score_result: {}".format(precision_score_result))
#     print("recall_score_result: {}".format(recall_score_result))
#     print("f1_score_result: {}".format(f1_score_result))
    
    return xg_model

# 数据类别
def check_columns_classifier(df, x_list):
    col_name_list = x_list
    classifier_list = []
    for i, name in enumerate(col_name_list):
        name_sum = df[name].value_counts().shape[0]
        classifier_list.append(name_sum)

    result_dict = {k: v for k, v in zip(col_name_list, classifier_list)}
    for rd in result_dict.items():
        print(rd)
    return None

In [3]:
# 读取数据
df_train = pd.read_csv("data/train.csv", low_memory=False)
df_train_label = pd.read_csv("data/train_label.csv", low_memory=False)
df_test = pd.read_csv("data/test.csv", low_memory=False)

# 查看标签值的比例
_ = check_classifier(df_train_label, col_name="Label")

# 分析行缺失比例
print("Train === 行缺失比例")
row_null_df_train = check_row_nan_count(df_train)
print("Test === 行缺失比例")
row_null_df_test = check_row_nan_count(df_test)

{0: 13425, 1: 941}
1:0 => 941:13425
比例: 14.3
Train === 行缺失比例
rate  count
0.00  0          16
0.01  2        1183
      1         327
0.02  3        2879
      4        1267
0.03  5         271
      6          60
0.04  7          14
0.08  15          2
0.09  16          2
0.10  17          1
      18          1
0.88  157       369
      156        29
0.89  159      3831
      158      1501
0.90  160      1456
      161       310
0.91  162        75
0.92  163        10
0.96  171       439
0.97  172       185
      173        94
0.98  174        44
Name: count, dtype: int64
Test === 行缺失比例
rate  count
0.00  0          21
0.01  2         812
      1         224
0.02  3        1878
      4         844
0.03  5         161
      6          51
0.04  7           7
0.08  15          1
0.09  16          2
0.88  157       281
      156        10
0.89  159      2582
      158      1002
0.90  160       940
      161       209
0.91  162        59
0.92  163         3
0.96  171       273
0.97  172     

In [4]:
# 新增特征：行缺失值数量，训练集Y标签字段
df_train["row_null_count"] = row_null_df_train["count"]
df_train["Y"] = df_train_label["Label"]

df_test["row_null_count"] = row_null_df_test["count"]
df_test["Y"] = -1

df = pd.concat([df_train.assign(is_train = 1),df_test.assign(is_train = 0)]) #合并train和test，并且用is_train进行标记
df.head(5)

,ID,企业类型,经营期限至,登记机关,企业状态,邮政编码,投资总额,注册资本,核准日期,行业代码,注销时间,经营期限自,成立日期,行业门类,企业类别,管辖机关,经营范围,增值税,企业所得税,印花税,教育费,城建税,递延收益_年初数,长期负债合计_年初数,长期借款_年初数,长期应付款_年初数,短期借款_年初数,递延所得税负债_年初数,非流动负债合计_年初数,负债合计_年初数,负债和所有者权益总计_年初数,交易性金融负债_年初数,流动负债合计_年初数,其他非流动负债_年初数,其他负债（或长期负债）_年初数,其他流动负债_年初数,其他应付款_年初数,其他应交款_年初数,实收资本（股本）_年初数,实收资本（或股本）净额_年初数,所有者权益合计_年初数,未分配利润_年初数,小企业_应付利润_年初数,应付福利费_年初数,应付股利_年初数,应付利息_年初数,应付票据_年初数,应付职工薪酬_年初数,应付债券_年初数,应付账款_年初数,预计负债_年初数,应交税费_年初数,一年内到期的非流动负债_年初数,预提费用_年初数,预收款项_年初数,盈余公积_年初数,资本公积_年初数,专项应付款_年初数,递延收益_年末数,长期负债合计_年末数,长期借款_年末数,长期应付款_年末数,短期借款_年末数,递延所得税负债_年末数,非流动负债合计_年末数,负债合计_年末数,负债和所有者权益总计_年末数,交易性金融负债_年末数,流动负债合计_年末数,其他非流动负债_年末数,其他负债（或长期负债）_年末数,其他流动负债_年末数,其他应付款_年末数,其他应交款_年末数,实收资本（股本）_年末数,实收资本（或股本）净额_年末数,所有者权益合计_年末数,未分配利润_年末数,小企业_应付利润_年末数,应付福利费_年末数,应付股利_年末数,应付利息_年末数,应付票据_年末数,应付职工薪酬_年末数,应付债券_年末数,应付账款_年末数,预计负债_年末数,应交税费_年末数,一年内到期的非流动负债_年末数,预提费用_年末数,预收款项_年末数,盈余公积_年末数,资本公积_年末数,专项应付款_年末数,货币资金_年初数,存货_年初数,存货中的原材料_年初数,存货中的周转材料_年初数,长期待摊费用_年初数,长期股权投资_年初数,长期投资合计_年初数,长期应收款_年初数,持有至到期投资_年初数,短期投资_年初数,待摊费用_年初数,递延所得税资产_年初数,非流动资产合计_年初数,工程物资_年初数,固定资产合计_年初数,固定资产净额_年初数,固定资产净值_年初数,固定资产清理_年初数,固定资产原价_年初数,交易性金融资产_年初数,库存商品(产成品)_年初数,开发支出_年初数,可供出售金融资产_年初数,流动资产合计_年初数,其他非流动资产_年初数,其他流动资产_年初数,其他应收款_年初数,生产性生物资产_年初数,商誉_年初数,投资性房地产_年初数,无形资产_年初数,无形资产及其他资产合计_年初数,应收账款_年初数,一年内到期的非流动资产_年初数,油气资产_年初数,应收补贴款_年初数,应收股利_年初数,应收利息_年初数,应收票据_年初数,预付款项_年初数,资产总计_年初数,在建工程_年初数,货币资金_年末数,存货_年末数,存货中的原材料_年末数,存货中的周转材料_年末数,长期待摊费用_年末数,长期股权投资_年末数,长期投资合计_年末数,长期应收款_年末数,持有至到期投资_年末数,短期投资_年末数,待摊费用_年末数,递延所得税资产_年末数,非流动资产合计_年末数,工程物资_年末数,固定资产合计_年末数,固定资产净额_年末数,固定资产净值_年末数,固定资产清理_年末数,固定资产原价_年末数,交易性金融资产_年末数,库存商品(产成品)_年末数,开发支出_年末数,可供出售金融资产_年末数,流动资产合计_年末数,其他非流动资产_年末数,其他流动资产_年末数,其他应收款_年末数,生产性生物资产_年末数,商誉_年末数,投资性房地产_年末数,无形资产_年末数,无形资产及其他资产合计_年末数,应收账款_年末数,一年内到期的非流动资产_年末数,油气资产_年末数,应收补贴款_年末数,应收股利_年末数,应收利息_年末数,应收票据_年末数,预付款项_年末数,资产总计_年末数,在建工程_年末数,row_null_count,Y,is_train
0,1,9.0,NaN,1.0,0.0,266300,19.110000,19.110000,00:00.0,5154.0,NaN,00:00.0,00:00.0,1.0,0.0,1.0,"[1412, 1024, 21375, 15023, 18526, 18789, 17022...",106.039650,0.331552,1.247218,3.181189,7.422776,0.0,0.0,25.110484,0.0,0.0,0.0,25.110484,797.386637,844.162546,0.0,772.276153,0.0,0.0,0.0,106.078436,0.0,50.220972,0.0,46.775910,-3.445063,0.0,0.0,0.0,0.0,125.552431,0.000000,0.0,520.839979,0.0,19.805307,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,585.013852,632.032182,0.0,585.013852,0.0,0.0,0.0,83.123866,0.0,50.220972,0.0,47.018330,-3.202643,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,489.934244,0.0,11.955743,0.0,0.0,0.000000,0.0,0.0,0.0,24.579246,462.290647,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,135.081011,0.0,135.081011,0.0,0.0,0.0,368.329907,0.0,0.0,0.0,0.0,709.081535,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,215.371727,0.0,0.0,0.0,0.0,0.0,2.647788,4.192126,844.162546,0.0,31.416284,229.147518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.128884,0.0,0.0,99.813781,0.0,99.813781,0.0,0.0,0.0,368.329907,0.0,0.0,0.0,0.0,532.218401,0.0,0.0,20.088389,0.0,0.0,0.0,0.0,0.0,142.310005,0.0,0.0,0.0,0.0,0.0,28.122902,1.004419,632.032182,0.0,2,0,1
1,3,11.0,NaN,5.0,0.0,266000,29.154194,19.210446,00:00.0,7214.0,NaN,31:40.0,31:40.0,0.0,2.0,3.0,"[3671, 24856, 13525, 21079, 13219, 13694, 1442...",39.175600,51.244655,0.014363,1.175269,2.742293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,158,0,1
2,9,0.0,NaN,3.0,0.0,266000,NaN,19.120045,13:31.0,2110.0,NaN,13:31.0,13:31.0,2.0,0.0,4.0,"[3671, 14394, 23662, 11227, 19205, 15633, 5344...",0.000000,0.000000,0.541067,4.637491,10.060346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [5]:
nd = check_nan(df)

(23944, 181)
其他应付款_年末数: 58.14%
工程物资_年末数: 58.14%
无形资产及其他资产合计_年末数: 58.14%
在建工程_年初数: 58.14%
长期应付款_年初数: 58.14%
应收票据_年末数: 58.14%
印花税: 9.32%
应付票据_年初数: 58.14%
应收补贴款_年初数: 58.14%
负债合计_年初数: 58.14%
一年内到期的非流动资产_年末数: 58.14%
实收资本（股本）_年末数: 58.14%
资本公积_年初数: 58.14%
小企业_应付利润_年初数: 58.14%
投资性房地产_年初数: 58.14%
无形资产及其他资产合计_年初数: 58.14%
其他非流动负债_年末数: 58.14%
工程物资_年初数: 58.14%
其他流动负债_年末数: 58.14%
经营期限至: 80.2%
开发支出_年末数: 58.14%
盈余公积_年初数: 58.14%
流动资产合计_年初数: 58.14%
存货_年初数: 58.14%
交易性金融负债_年初数: 58.14%
生产性生物资产_年初数: 58.14%
固定资产净值_年末数: 58.14%
递延所得税资产_年初数: 58.14%
一年内到期的非流动资产_年初数: 58.14%
应收账款_年末数: 58.14%
应付股利_年初数: 58.14%
存货中的原材料_年末数: 58.14%
货币资金_年末数: 58.14%
应付债券_年初数: 58.14%
固定资产原价_年初数: 58.14%
应收账款_年初数: 58.14%
长期负债合计_年初数: 58.14%
应收利息_年末数: 58.14%
预提费用_年末数: 58.14%
递延所得税资产_年末数: 58.14%
油气资产_年初数: 58.14%
负债和所有者权益总计_年末数: 58.14%
其他非流动资产_年初数: 58.14%
短期投资_年初数: 58.14%
投资性房地产_年末数: 58.14%
油气资产_年末数: 58.14%
应付福利费_年末数: 58.14%
一年内到期的非流动负债_年末数: 58.14%
应交税费_年初数: 58.14%
负债合计_年末数: 58.14%
预计负债_年末数: 58.14%
专项应付款_年末数: 58.14%
非流动资产合计_年末数: 58.14%
长期股权投资

In [6]:
check_columns_classifier(df, ["邮政编码", "经营范围", "登记机关", "管辖机关", "企业状态", "行业门类", "企业类别", "行业代码", "企业类型"])
# df["总交税"] = df["增值税"] + df["企业所得税"] + df["印花税"] + df["城建税"] + df["教育费"]

# 列缺失是否严重
df.loc[df.row_null_count <= 18,'列缺失超过88的数据'] = 0
df.loc[df.row_null_count > 18,'列缺失超过88的数据'] = 1
df = df.drop(["row_null_count"], axis=1)

# 时间处理
df = tran_date(df, "成立日期")
bus_count = []
for d in df["成立日期"]:
    bus_count.append(d)
print("成立日期 MAX:", max(bus_count))

# 经营范围数量
bus_count = []
for d in df["经营范围"]:
    d = eval(d)
    c = len(d)
    bus_count.append(c)
df["经营范围数量"] = bus_count

df = df.reset_index(drop=True)
aptmt_set = set()
for i in range(df.shape[0]):
    v = "{}_{}".format(df["登记机关"][i], df["管辖机关"][i])
    aptmt_set.add(v)

# aptmt_set_dict = dict()
# for index, value in enumerate(aptmt_set):
#     aptmt_set_dict[value] = index

# f_list = []
# for i in range(df.shape[0]):
#     v = "{}_{}".format(df["登记机关"][i], df["管辖机关"][i])
#     f = aptmt_set_dict[v]
#     f_list.append(f)

# df["登记机关_管辖机关"] = f_list

df = df.drop(["注销时间", "核准日期", "经营期限至", "经营期限自"], axis=1)

df = df.drop(["邮政编码", "经营范围", "登记机关", "管辖机关"], axis=1)

('管辖机关', 13)
('企业状态', 4)
('登记机关', 13)
('企业类型', 84)
('经营范围', 8319)
('行业代码', 693)
('邮政编码', 278)
('企业类别', 4)
('行业门类', 20)
成立日期 MAX: 3800.0


In [7]:
nd = check_nan(df)

# 离散变量
scatter_fields = ["企业状态", "行业门类", "企业类别", "行业代码", "企业类型"]  # 行业代码
# 资本和投资数据
fill_small_fields = ["投资总额", "注册资本"]
# 几个重要的税值
important_price_fields = ["增值税", "企业所得税", "印花税", "城建税", "教育费"]

# 填充-999
no_continuous_fields = scatter_fields + fill_small_fields + important_price_fields
continuous_fields = []
for n in nd.keys():
    if n in no_continuous_fields:
        pass
    else:
        continuous_fields.append(n)

df = df.reset_index(drop=True)

# mean median most_frequent constant
# 离散变量-填充平均值
df = imputer_nan(df, axis=1, cols=scatter_fields, missing_values="NaN", strategy='constant', fill_value=-1)

# 填充中位数
# df = imputer_nan(df, axis=1, cols=[], missing_values="NaN", strategy='median', fill_value=0)

# 几个重要的税值-填充平均值
df = imputer_nan(df, axis=1, cols=important_price_fields, missing_values="NaN", strategy='mean', fill_value=-999)

# 资本和投资数据-连续变量-填充常量 -999
df = imputer_nan(df, axis=1, cols=fill_small_fields, missing_values="NaN", strategy='constant', fill_value=-999)
df = imputer_nan(df, axis=1, cols=continuous_fields, missing_values="NaN", strategy='constant', fill_value=-999)


nd = check_nan(df)

(23944, 174)
其他应付款_年末数: 58.14%
工程物资_年末数: 58.14%
无形资产及其他资产合计_年末数: 58.14%
在建工程_年初数: 58.14%
长期应付款_年初数: 58.14%
应收票据_年末数: 58.14%
印花税: 9.32%
应付票据_年初数: 58.14%
应收补贴款_年初数: 58.14%
一年内到期的非流动资产_年末数: 58.14%
实收资本（股本）_年末数: 58.14%
资本公积_年初数: 58.14%
小企业_应付利润_年初数: 58.14%
应收股利_年末数: 58.14%
无形资产及其他资产合计_年初数: 58.14%
其他非流动负债_年末数: 58.14%
工程物资_年初数: 58.14%
其他流动负债_年末数: 58.14%
开发支出_年末数: 58.14%
盈余公积_年初数: 58.14%
流动资产合计_年初数: 58.14%
存货_年初数: 58.14%
交易性金融负债_年初数: 58.14%
生产性生物资产_年初数: 58.14%
固定资产净值_年末数: 58.14%
递延所得税资产_年初数: 58.14%
一年内到期的非流动资产_年初数: 58.14%
负债和所有者权益总计_年初数: 58.14%
应付股利_年初数: 58.14%
存货中的原材料_年末数: 58.14%
货币资金_年末数: 58.14%
应付债券_年初数: 58.14%
固定资产原价_年初数: 58.14%
应收账款_年初数: 58.14%
应收利息_年末数: 58.14%
预提费用_年末数: 58.14%
递延所得税资产_年末数: 58.14%
油气资产_年初数: 58.14%
负债和所有者权益总计_年末数: 58.14%
其他非流动资产_年初数: 58.14%
短期投资_年初数: 58.14%
投资性房地产_年末数: 58.14%
油气资产_年末数: 58.14%
应付福利费_年末数: 58.14%
一年内到期的非流动负债_年末数: 58.14%
应交税费_年初数: 58.14%
负债合计_年末数: 58.14%
预计负债_年末数: 58.14%
专项应付款_年末数: 58.14%
非流动资产合计_年末数: 58.14%
商誉_年初数: 58.14%
持有至到期投资_年初数: 58.14%
其他流动资产_年末数: 58.1

In [8]:
# 重要税值的比值，会有两种情况
# 0/0 => NaN
# constant/0 => inf
# 0/constant => 0
important_price_fields = ["增值税", "企业所得税", "印花税", "城建税", "教育费", "经营范围数量"]

inf = np.inf
nan = np.nan

for index1, f1 in enumerate(important_price_fields):
    for index2, f2 in enumerate(important_price_fields):
        if index2 > index1:
            l_value_list = []
            for index, a in enumerate(df[f1]):
                b = df[f2][index]

                if a == b == 0:
                    value = -999
                elif b ==0 and a > 0:
                    value = 999
                else:
                    value = a / b
                l_value_list.append(value)

            df["{}_{}_比值".format(f1, f2)] = l_value_list


In [9]:
dl_list = ["行业门类", "企业状态", "企业类别", "企业类型"]
df = OnehotEncoding(df, columns=dl_list)
df = df.drop(dl_list, axis=1)

In [10]:
# 5, 8, 11, 15, 18, 21, 25, 30, 33, 38, 41
for ne in [1]:
    df_train = df[df["is_train"] == 1]
    df_test = df[df["is_train"] == 0]

    df_train = df_train.drop(["ID", "is_train"], axis=1)

    df_train_train, df_train_valid = split_train_test(df_train, random_state=15, test_size=0.25)

    df_train_train_x, df_train_train_y = split_column(df_train_train, y="Y")
    df_train_valid_x, df_train_valid_y = split_column(df_train_valid, y="Y")

    df_train.head(30)
    
    model = train_and_valid(df_train_train_x, df_train_train_y, df_train_valid_x, df_train_valid_y)
    # 计算y_prob, y_pred
    y_prob = model.predict_proba(df_train_valid_x)
    # 计算auc, ks, gini
    fpr, tpr, threshold = metrics.roc_curve(df_train_valid_y, y_prob[:, 1])
    auc_value = metrics.auc(fpr, tpr)  # 计算auc
    print("=======", ne, "** auc_value: 【{}】 **".format(auc_value))
    
result_dict = {k: v for k, v in zip(df_train_train_x.columns.values, list(model.feature_importances_))}
dict_list = sorted(result_dict.items(), key=lambda x: x[1], reverse=True)
not_zero_list = []
for dl in dict_list:
    if dl[1] < 0.01:
        not_zero_list.append(dl[0])
    else:
        print(dl)
# print(not_zero_list)

# 固定资产原价
# 负债和所有者权益总计
# 流动资产合计
# 货币资金
# 非流动资产合计
# 预收款项

======= 1 ** auc_value: 【0.9339420892336016】 **
('城建税_教育费_比值', 0.06350504)
('成立日期', 0.05042246)
('经营范围数量', 0.04905969)
('行业代码', 0.04905969)
('印花税_经营范围数量_比值', 0.046606705)
('印花税', 0.043881167)
('企业所得税', 0.04197329)
('增值税_城建税_比值', 0.04115563)
('注册资本', 0.039520305)
('增值税', 0.03924775)
('企业所得税_经营范围数量_比值', 0.038157538)
('城建税', 0.02916326)
('增值税_教育费_比值', 0.028345598)
('城建税_经营范围数量_比值', 0.028345598)
('企业所得税_印花税_比值', 0.027800491)
('增值税_经营范围数量_比值', 0.027527938)
('增值税_印花税_比值', 0.022349413)
('印花税_城建税_比值', 0.020168982)
('增值税_企业所得税_比值', 0.017443445)
('教育费', 0.017443445)
('投资总额', 0.017443445)
('印花税_教育费_比值', 0.016625784)
('企业所得税_城建税_比值', 0.014717907)
('货币资金_年末数', 0.012264922)
('货币资金_年初数', 0.012264922)
('x3_0.0', 0.010357046)


In [11]:
df_test, df_id = split_column(df_test, y="ID")
df_test = df_test.drop(["Y", "is_train"], axis=1)
df_result = model.predict(df_test)
df_id["Label"] = df_result
_ = check_classifier(df_id, col_name="Label")

{0: 9389, 1: 189}
1:0 => 189:9389
比例: 49.7


In [12]:
df_result = model.predict_proba(df_test)

dr_list = []
for dr in df_result:
    dr_list.append(dr[1])

df_id["Label"] = dr_list

_ = df_id.to_csv('Result_{}.csv'.format(str(auc_value)), index=0)

In [13]:
print("** auc_value: {} **".format(auc_value))

** auc_value: 0.9339420892336016 **


In [14]:


# ** auc_value: 0.9272014353458055 ** 不删掉 "登记机关", "管辖机关" 准确率更高
# 使用影响度高的其他税值，有变高的倾向，但是过多就会变低，说明需要挑选有用的放上去才可以

# ======= n_e 88 ** auc_value: 【0.9282555347133458】 **

# drop dl_list = ["列缺失超过88的数据", "是否注销"]  ** auc_value: 0.9282735180358892 ** random_state 33

# random_state
# 33 0.92827 最开始是33
# 3  0.9231
# 15 0.93084  榜上 0.928  经营范围数量(+) 加上后榜上下降到 0.927
# 41 0.933    榜上过拟合

# 经营范围数量(+) + random_states 15  ==> 0.93288
# 经营范围数量(-) + random_states 15  ==> 0.93084
# 企业类型(onehot -) + random_states 15  ==> 0.93291
# 企业类型(-) + random_states 15  ==> 0.93352


# 经营范围降维，成立日期、注册资本、几个税费做分箱

# 0.93352
# 0.93400
# 0.93436
# 0.93466
# 0.93501